## Deploy a model package with CloudFormation

In [ ]:
import boto3
import sagemaker

sm = boto3.client('sagemaker')
cf = boto3.client('cloudformation')

role = sagemaker.get_execution_role()

## Set up endpoint properties

In [ ]:
model1_package_arn = 'MODEL_PACKAGE_ARN'
model1_name = 'MODEL_NAME'

In [ ]:
response = sm.describe_model_package(ModelPackageName=model1_package_arn)

training_image = response['InferenceSpecification']['Containers'][0]['Image']
model1_path     = response['InferenceSpecification']['Containers'][0]['ModelDataUrl']

print(training_image)
print(model1_path)

## Create endpoint

In [ ]:
with open('endpoint-one-model.yml', 'r') as f:
	response = cf.create_stack(
        StackName='endpoint-one-model', 
		TemplateBody=f.read(),
		Parameters=[
			{'ParameterKey':'ModelName',     'ParameterValue': model1_name},
			{'ParameterKey':'TrainingImage', 'ParameterValue': training_image},
			{'ParameterKey':'ModelDataUrl',  'ParameterValue': model1_path},
			{'ParameterKey':'RoleArn',       'ParameterValue': role}
        ]
    )
    
print(response)

In [ ]:
waiter = cf.get_waiter('stack_create_complete')
waiter.wait(StackName='endpoint-one-model')

## Create a change set to add a second variant

In [ ]:
model2_package_arn = 'MODEL2_PACKAGE_ARN'
model2_name = 'MODEL2_NAME'

In [ ]:
response = sm.describe_model_package(ModelPackageName=model2_package_arn)

model2_path = response['InferenceSpecification']['Containers'][0]['ModelDataUrl']

print(model2_path)

In [ ]:
with open('endpoint-two-models.yml', 'r') as f:
    response = cf.create_change_set(
        StackName='endpoint-one-model', 
        ChangeSetName='add-production-variant',
        TemplateBody=f.read(),
        Parameters=[
            {'ParameterKey':'ModelName',           'UsePreviousValue': True},
			{'ParameterKey':'TrainingImage',       'UsePreviousValue': True},
			{'ParameterKey':'ModelDataUrl',        'UsePreviousValue': True},
			{'ParameterKey':'RoleArn',             'UsePreviousValue': True},
			{'ParameterKey':'ModelName2',          'ParameterValue': model2_name},
            {'ParameterKey':'ModelDataUrl2',       'ParameterValue': model2_path},
            {'ParameterKey':'ModelVariantWeight',  'ParameterValue': str(95)},
            {'ParameterKey':'ModelVariantWeight2', 'ParameterValue': str(5)},
        ]
    )
    
print(response)

In [ ]:
response = cf.execute_change_set(
    ChangeSetName='add-production-variant',
    StackName='endpoint-one-model'
)

print(response)

In [ ]:
waiter = cf.get_waiter('stack_update_complete')
waiter.wait(StackName='endpoint-one-model')

In [ ]:
response = cf.delete_stack(StackName='endpoint-one-model')